In [1]:
import pickle
import pandas as pd

# Load the pickled model
with open('pkl_models/ABC_model.pkl', 'rb') as model_file_ABC:
    modelABC = pickle.load(model_file_ABC)
with open('pkl_models/DTC_model.pkl', 'rb') as model_file_DTC:
    modelDTC = pickle.load(model_file_DTC)
with open('pkl_models/KNN_model.pkl', 'rb') as model_file_KNN:
    modelKNN = pickle.load(model_file_KNN)
with open('pkl_models/RFC_model.pkl', 'rb') as model_file_KNN:
    modelRFC = pickle.load(model_file_KNN)
with open('pkl_models/XGB_model.pkl', 'rb') as model_file_XGB:
    modelXGB = pickle.load(model_file_XGB)

In [2]:
df=pd.read_csv('ratio_dataset.csv')
df = df.drop(columns=['step','nameOrig', 'nameDest','isFlaggedFraud'])
df.head()

,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
0,PAYMENT,9839.64,170136.0,160296.36,0.0,0.0,0
1,PAYMENT,1864.28,21249.0,19384.72,0.0,0.0,0
2,TRANSFER,181.00,181.0,0.00,0.0,0.0,1
3,CASH_OUT,181.00,181.0,0.00,21182.0,0.0,1
4,PAYMENT,11668.14,41554.0,29885.86,0.0,0.0,0


In [3]:
df['type']=df['type'].map({'PAYMENT':2, 'TRANSFER':4, 'CASH_OUT':1, 'DEBIT':5, 'CASH_IN':3})
x=df[['type','amount','oldbalanceOrg','newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']]
y=df.iloc[:,-1]

In [4]:
from sklearn.model_selection import train_test_split


x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [6]:
from sklearn.metrics import accuracy_score
import xgboost as xgb
accABC = modelABC.score(x_test, y_test)*100
accDTC = modelDTC.score(x_test, y_test)*100
accKNN = modelKNN.score(x_test, y_test)*100
accRFC = modelRFC.score(x_test, y_test)*100
dmtest = xgb.DMatrix(x_test)
y_pred_XGB = modelXGB.predict(dmtest)
accXGB = accuracy_score(y_test, y_pred_XGB)*100

print("DecisionTree:",accDTC)
print("AdaBoost:",accABC)
print("KNN:",accKNN)
print("RandomForest:",accRFC)
print("XGBoost:",accXGB)

DecisionTree: 99.48320413436691
AdaBoost: 99.48320413436691
KNN: 99.2248062015504
RandomForest: 99.6124031007752
XGBoost: 99.35400516795866


In [7]:
user_input = {
    'type': 4,  # This should be a number representing the transaction type
    'amount': 181,  # The amount of the transaction
    'oldbalanceOrg': 181,  # The original balance before the transaction
    'newbalanceOrig': 0,  # The new balance after the transaction
    'oldbalanceDest': 0,  # The original balance of the recipient before the transaction
    'newbalanceDest': 0  # The new balance of the recipient after the transaction
}
new_data_df = pd.DataFrame([user_input])
new_data_df.values

array([[  4, 181, 181,   0,   0,   0]], dtype=int64)

In [8]:
predABC = modelABC.predict(new_data_df)
predDTC = modelDTC.predict(new_data_df)
predKNN = modelKNN.predict(new_data_df)
predRFC = modelRFC.predict(new_data_df)

dtest = xgb.DMatrix(new_data_df)
predXGB = modelXGB.predict(dtest)

pred_list = [predABC[0],predDTC[0],predKNN[0],predRFC[0],predXGB[0]]

count_0 = pred_list.count(0)
count_1 = pred_list.count(1)

is_fraud = count_0 < count_1

is_fraud

True